In [335]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE, RandomOverSampler
from xgboost import XGBClassifier
import warnings
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from itertools import product
from sklearn.metrics import classification_report
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler

#For reproducibility
np.random.seed(42)

#Load the data
file_path = r"C:\Users\20201527\OneDrive - TU Eindhoven\Desktop\8CC00\Group assignement\final_pca_dataset.csv"
descriptor_df = pd.read_csv(file_path)
descriptor_df['PKM2_inhibition'] = data['PKM2_inhibition']
descriptor_df['ERK2_inhibition'] = data['ERK2_inhibition']
descriptor_df

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,PKM2_inhibition,ERK2_inhibition
0,-15.071416,-0.901064,-0.582650,-0.900137,-4.259277,-2.008358,1.932322,2.483424,0.896982,3.631180,...,1.197561,0.654058,0.770409,0.552815,-0.355040,-1.511947,0.124026,0.474687,0,0
1,-8.946369,-0.583776,-1.738102,-0.381471,-0.084150,-0.017619,-1.771762,-0.248554,-2.019284,-1.654706,...,0.104914,-0.831474,-0.066544,-0.184380,0.027305,0.935941,-0.530785,0.216484,0,0
2,-8.995540,4.154783,-1.992762,-0.148727,-2.481074,0.042751,3.991139,-4.616611,5.123222,0.552886,...,-0.583351,0.400289,1.147787,0.454078,-0.556067,-0.134297,-0.870352,0.505478,0,0
3,-8.502632,-0.513989,-2.155378,-1.095535,-0.499122,1.857929,0.470431,1.819208,0.902396,-2.166603,...,0.312112,-0.996122,0.431695,0.548602,0.028028,-0.347224,-0.780778,0.833347,0,0
4,-10.864690,0.399758,-1.840009,3.154235,0.008332,0.290368,2.191179,-0.727996,0.508694,-2.502573,...,0.308007,-0.093068,0.344138,0.008183,0.243019,-0.732514,-0.839053,-0.578841,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,2.168633,-1.376467,0.204709,-5.357065,1.680453,0.749252,-2.305554,0.055191,-2.551296,-1.077388,...,-1.715754,-1.381099,-0.802750,0.263426,-0.149543,0.322965,0.585168,-0.038369,0,1
1112,-0.012814,-4.280241,-2.194752,2.517053,5.088990,-3.540141,0.557611,2.827925,0.614404,2.372873,...,-0.508355,0.155767,0.162331,0.358323,-0.252211,-0.465345,-0.439143,-0.361104,0,1
1113,0.581355,-1.893748,4.327930,-2.874801,-2.447483,-2.503895,-0.730804,-1.051563,-3.129562,0.243655,...,-0.529850,0.660782,0.707800,-0.756614,-0.697578,0.137095,0.202787,0.161281,0,1
1114,-2.676144,-5.542960,5.364963,-1.346702,1.687476,-4.348146,0.203293,-0.551945,-1.264552,-3.288822,...,-0.272840,-0.561771,0.182573,-1.130903,0.679011,-0.199342,-0.666061,0.757977,0,1


In [336]:
# Assuming your descriptor_df is already prepared
X = descriptor_df.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])
y1 = descriptor_df['PKM2_inhibition']
y2 = descriptor_df['ERK2_inhibition']
y = pd.concat([y1, y2], axis=1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [337]:
# Ignore all warnings
warnings.filterwarnings("ignore")

# Apply RandomOverSampler to handle imbalance for each target separately
ros_pkm2 = RandomOverSampler(random_state=42)
X_train_ros_pkm2, y_train_ros_pkm2 = ros_pkm2.fit_resample(X_train, y_train['PKM2_inhibition'])

ros_erk2 = RandomOverSampler(random_state=42)
X_train_ros_erk2, y_train_ros_erk2 = ros_erk2.fit_resample(X_train, y_train['ERK2_inhibition'])

# Calculate scale_pos_weight for each target
scale_pos_weight_pkm2 = y_train_ros_pkm2.value_counts()[0] / y_train_ros_pkm2.value_counts()[1]
scale_pos_weight_erk2 = y_train_ros_erk2.value_counts()[0] / y_train_ros_erk2.value_counts()[1]

# Create and train the XGBClassifier for PKM2_inhibition
model_pkm2 = XGBClassifier(scale_pos_weight=scale_pos_weight_pkm2, random_state=42)
model_pkm2.fit(X_train_ros_pkm2, y_train_ros_pkm2)

# Create and train the XGBClassifier for ERK2_inhibition
model_erk2 = XGBClassifier(scale_pos_weight=scale_pos_weight_erk2, random_state=42)
model_erk2.fit(X_train_ros_erk2, y_train_ros_erk2)

# Predict on the validation set
y_pred_pkm2 = model_pkm2.predict(X_val)
y_pred_erk2 = model_erk2.predict(X_val)

# Convert y_val to numpy array for comparison
y_val_binary = y_val.to_numpy()

# Calculate the accuracy, precision, recall, F1 score, and balanced accuracy for each target column
accuracy_pkm2 = accuracy_score(y_val_binary[:, 0], y_pred_pkm2)
accuracy_erk2 = accuracy_score(y_val_binary[:, 1], y_pred_erk2)

precision_pkm2 = precision_score(y_val_binary[:, 0], y_pred_pkm2)
precision_erk2 = precision_score(y_val_binary[:, 1], y_pred_erk2)

recall_pkm2 = recall_score(y_val_binary[:, 0], y_pred_pkm2)
recall_erk2 = recall_score(y_val_binary[:, 1], y_pred_erk2)

f1_pkm2 = f1_score(y_val_binary[:, 0], y_pred_pkm2)
f1_erk2 = f1_score(y_val_binary[:, 1], y_pred_erk2)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'PKM2_inhibition': [accuracy_pkm2, precision_pkm2, recall_pkm2, f1_pkm2],
    'ERK2_inhibition': [accuracy_erk2, precision_erk2, recall_erk2, f1_erk2]
})

# Transpose the DataFrame for the desired format
results = results.set_index('Metric').T

results

Metric,Accuracy,Precision,Recall,F1 Score
PKM2_inhibition,0.976703,0.50,0.307692,0.380952
ERK2_inhibition,0.949821,0.25,0.038462,0.066667


In [338]:
# Print the classification reports
print("Classification report for PKM2_inhibition:")
print(classification_report(y_val_binary[:, 0], y_pred_pkm2))

print("Classification report for ERK2_inhibition:")
print(classification_report(y_val_binary[:, 1], y_pred_erk2))

# Combine the predictions for both targets
y_pred_combined = pd.concat([pd.Series(y_pred_pkm2, name='PKM2_inhibition'), pd.Series(y_pred_erk2, name='ERK2_inhibition')], axis=1)
print("Predicted binary values:\n", y_pred_combined)

Classification report for PKM2_inhibition:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       545
           1       0.50      0.31      0.38        13

    accuracy                           0.98       558
   macro avg       0.74      0.65      0.68       558
weighted avg       0.97      0.98      0.97       558

Classification report for ERK2_inhibition:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       532
           1       0.25      0.04      0.07        26

    accuracy                           0.95       558
   macro avg       0.60      0.52      0.52       558
weighted avg       0.92      0.95      0.93       558

Predicted binary values:
      PKM2_inhibition  ERK2_inhibition
0                  0                0
1                  0                0
2                  0                0
3                  0                0
4                  0                0
..    

In [339]:
# Create a new DataFrame with all possible combinations
all_combinations = pd.DataFrame(list(product([0, 1], [0, 1])), columns=['PKM2_inhibition', 'ERK2_inhibition'])
all_combinations.set_index(['PKM2_inhibition', 'ERK2_inhibition'], inplace=True)

# Count the occurrences of each combination in the predictions
sequence_counts = y_pred_combined.value_counts().reset_index(name='Count')
sequence_counts.set_index(['PKM2_inhibition', 'ERK2_inhibition'], inplace=True)

# Reindex to ensure all combinations are present, filling missing values with 0
sequence_counts = all_combinations.join(sequence_counts, how='left').fillna(0).astype(int)

# Reset the index to turn PKM2_inhibition and ERK2_inhibition back into columns
sequence_counts.reset_index(inplace=True)


sequence_counts

,PKM2_inhibition,ERK2_inhibition,Count
0,0,0,546
1,0,1,4
2,1,0,8
3,1,1,0
